# FastMCP

`FastMCP` is the **standard framework** for working with the **Model Context Protocol**. `FastMCP 1.0` was incorporated into the official MCP Python SDK in 2024.
`FastMCP 2.0`, is the **actively maintained version** that provides a complete toolkit for working with the **MCP ecosystem** including client libraries, authentication systems, deployment tools, integrations with major AI platforms, testing frameworks, and production-ready infrastructure patterns.

## MPC (Model Context Protocol)

The **Model Context Protocol** lets you build servers that expose data and functionality to LLM applications in a secure, standardized way.
MCP servers can:

* **Expose data** through `Resources` (think of these sort of like **GET endpoints**; they are used <u>to load information into the LLM’s context</u>)
* **Provide functionality** through `Tools` (sort of like **POST endpoints**; they are used <u>to execute code or otherwise produce a side effect</u>)
* **Define interaction patterns** through `Prompts` (reusable templates for LLM interactions)

FastMCP provides a high-level, `Pythonic interface` for **building**, **managing**, and **interacting** with these **servers**.
In addition, It handles all the complex protocol details and server management.

In [12]:
from fastmcp import FastMCP, Client
import asyncio, nest_asyncio
nest_asyncio.apply()

### Create a FastMCP Server


In [ ]:
# Create a FastMCP server
mcp = FastMCP("My MCP Server")

### Adding a tool to the server

In [5]:
# it uses the @mcp.tool decorator to add a new tool to the server
@mcp.tool
def greet(name: str) -> str:
    return f"Hello, {name}!"

### Testing the Server
To test the server, create a **FastMCP client** and point it at the **server object**.
* Clients are asynchronous, so we need to use `asyncio.run` to run the `client`.
* We must enter a client context (`async with client:`) before using the `client`. <u>You can make **multiple client calls** within the same context.</u>

`Note`: When we say that a client is asynchronous, we mean that it can perform multiple tasks without blocking or waiting for each other to complete. In the context of a `client-server` **architecture**, an **asynchronous client** <u>can send multiple requests to the server without waiting for the previous request to complete.</u>

In [ ]:
# Create a FastMCP client
client = Client(mcp)

### Making an asynchronous call to an MCP Server

"When running `asyncio.run()` in a Jupyter notebook, a RuntimeError occurs because the notebook already has an event loop running. To fix this, use `nest_asyncio` or run the code in a `standalone Python script` instead."
 
* The `asyncio` library is an standard of Python for async programming. It is used to write asynchronous code that runs concurrently, namely, that can realize several tasks at the same time without blocking the running of the program.

* The `nest_asyncio` library is used to run asynchronous  code within an existing event loop. It is useful when it want to execute asynchronous code within an enviroment that it has an event loop in execution, as in the case with Jupyter Notebook.

Notes: 


* **Concurrency** is the ability to handle multiple tasks at the same time, but not necessarily simultaneously. It means that tasks can start, run, and complete in overlapping time periods, often with a single processor.

`Example`: Imagine a single chef (CPU) preparing multiple dishes (tasks) at once. They might chop vegetables for one dish, then stir another, then start a third, and so on. The dishes aren't all being cooked at the exact same moment, but the chef is managing them all concurrently. In programming, this is achieved through context switching, where the CPU rapidly switches between different tasks

* **Parallelism** is the ability to execute multiple tasks simultaneously, often on different processor cores.

`Example`: Now imagine two chefs (two CPUs) each working on a different dish simultaneously. They are both cooking at the same time, using their own resources (stoves, pots). This is parallelism – tasks executing truly at the same time. In programming, this requires multiple processing units (cores) to run tasks concurrently.


*  **Asynchronous programming** is a technique that allows tasks to run without blocking the main program flow. In other words, allows a program to perform multiple operations concurrently without waiting for each operation to complete before starting the next.

`Example`: Think of a person ordering food at a restaurant. They place the order (initiate a task) and then can continue browsing their phone or talking to a friend (do other work) while the restaurant prepares the food. The person doesn't have to sit and stare at the kitchen waiting for the food. In programming, asynchronous operations allow a program to continue executing other code while waiting for a task (like fetching data from a database or a website) to complete. This prevents the program from becoming unresponsive. 

In [ ]:
# Define an async function to call a tool 
async def call_tool(name: str):
    async with client: #  is a context manager that ensures the client object is properly cleaned up after use.  
        result = await client.call_tool("greet", {"name": name}) # an asynchronous call to the call_tool method of the client object
        print(result) # prints the result of the call

asyncio.run(call_tool("Ford"))

RuntimeError: asyncio.run() cannot be called from a running event loop

### Solution 

In [21]:
# Define an async function to call a tool 
async def call_tool(name: str):
    async with client: #  is a context manager that ensures the client object is properly cleaned up after use.  
        result = await client.call_tool("greet", {"name": name}) # an asynchronous call to the call_tool method of the client object
        return(result) # prints the result of the call

await call_tool("Ford")

CallToolResult(content=[TextContent(type='text', text='Hello, Ford!', annotations=None, meta=None)], structured_content={'result': 'Hello, Ford!'}, data='Hello, Ford!', is_error=False)

In [ ]:
result = await call_tool("Maria")

In [31]:
print(result)

CallToolResult(content=[TextContent(type='text', text='Hello, Maria!', annotations=None, meta=None)], structured_content={'result': 'Hello, Maria!'}, data='Hello, Maria!', is_error=False)


In [23]:
print(type(result))

<class 'fastmcp.client.client.CallToolResult'>


In [34]:
# Access the content of the result
print(result.content[0].text)

# Access the annotations of the result
print(result.content[0].annotations)

Hello, Maria!
None


In [ ]:
# Access the structured content of the result
print(result.structured_content["result"])

Hello, Maria!


In [ ]:
# Access the data of the result
print(result.data)

Hello, Maria!
